In [1]:
from happytransformer import HappyTextToText

happy_tt = HappyTextToText("T5", "t5-base")


C:\ProgramData\Anaconda3\envs\pdp\Lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
11/16/2023 21:25:33 - INFO - happytransformer.happy_transformer -   Using device: cuda:0


In [16]:
from datasets import load_dataset

train_dataset = load_dataset("jfleg", split='validation[:]')

eval_dataset = load_dataset("jfleg", split='test[:]')

In [3]:
for case in train_dataset["corrections"][:2]:
  print(case)
  print(case[0])
  print("--------------------------------------------------------")


['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
--------------------------------------------------------
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
Not for use with a car . 
--------------------------------------------------------


In [4]:
for case in train_dataset["sentence"][:2]:
  print(case)
  print("--------------------------------------------------------")

So I think we can not live if old people could not find siences and tecnologies and they did not developped . 
--------------------------------------------------------
For not use car . 
--------------------------------------------------------


In [5]:
import csv

def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                if input_text and correction:
                    writter.writerow([input_text, correction])



generate_csv("train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)


In [6]:
before_result = happy_tt.eval("eval.csv")


11/16/2023 21:25:40 - INFO - happytransformer.happy_transformer -   Preprocessing dataset...


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [7]:
print("Before loss:", before_result.loss)


Before loss: 1.280392050743103


In [8]:
from happytransformer import TTTrainArgs

args = TTTrainArgs(batch_size=8)
happy_tt.train("train.csv", args=args)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

11/16/2023 21:27:00 - INFO - happytransformer.happy_transformer -   Tokenizing training data...


Map:   0%|          | 0/2714 [00:00<?, ? examples/s]

Map:   0%|          | 0/302 [00:00<?, ? examples/s]

11/16/2023 21:27:13 - INFO - happytransformer.happy_transformer -   Moving model to cuda:0


Step,Training Loss,Validation Loss
1,1.413500,1.479743
34,0.845100,0.688992
68,0.760400,0.585312
102,0.667900,0.548959
136,0.644200,0.534469
170,0.624000,0.526296
204,0.609400,0.512749
238,0.603500,0.505470
272,0.572000,0.502395
306,0.561500,0.499316


In [9]:
before_loss = happy_tt.eval("eval.csv")

print("After loss: ", before_loss.loss)


11/16/2023 21:32:48 - INFO - happytransformer.happy_transformer -   Preprocessing dataset...


Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

After loss:  0.48176899552345276


In [10]:
from happytransformer import TTSettings

beam_settings = TTSettings(num_beams=5, min_length=1, max_length=20)

In [11]:
example_1 = "grammar: This sentences, has bads grammar and spelling!"
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

11/16/2023 21:34:19 - INFO - happytransformer.happy_transformer -   Initializing a pipeline


This sentences, has bad grammar and spelling!


In [12]:
result_2 = happy_tt.generate_text("grammar: I need some helps", args=beam_settings)
print(result_2.text)

I need some help.


In [17]:
def write_row(writer, case):
    input_text = "grammar: " + case["sentence"]
    for correction in case["corrections"]:
        if input_text and correction:
            writer.writerow([input_text, correction])

with open('complete.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["input", "target"])
    for case in train_dataset:
        write_row(writer, case)
    for case in eval_dataset:
        write_row(writer, case)


In [19]:
final_model = HappyTextToText("T5", "t5-base")
final_model.train("complete.csv", args=args)

11/16/2023 21:47:39 - INFO - happytransformer.happy_transformer -   Using device: cuda:0


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

11/16/2023 21:47:40 - INFO - happytransformer.happy_transformer -   Tokenizing training data...


Map:   0%|          | 0/5403 [00:00<?, ? examples/s]

Map:   0%|          | 0/601 [00:00<?, ? examples/s]

11/16/2023 21:47:50 - INFO - happytransformer.happy_transformer -   Moving model to cuda:0
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1,2.016000,1.083065
68,0.767000,0.562111
136,0.619300,0.511633
204,0.551700,0.488605
272,0.559200,0.471309
340,0.530400,0.460891
408,0.518400,0.454917
476,0.533700,0.444905
544,0.537600,0.439670
612,0.534700,0.438502


In [20]:
final_model.save("model/")